# Comparing ascites vs tumour/healthy DC

## DC2 previously subclustered and found one cluster (cliser 6) enriched in ascites

In [ ]:
#load packages I need
import os
import tools
import scanpy as sc
import pandas as pd
import dandelion as ddl
from tqdm import tqdm
import matplotlib.pyplot as plt
import scanpy.external as sce
from matplotlib.pyplot import rc_context
import seaborn as sns

In [ ]:
#set current directory 
os.chdir('/scratch/user/s4436039/scdata/Myeloid_Objects')
os.getcwd()

In [ ]:
data_DC2 = sc.read_h5ad('20250629_DC2_bbknn_clustered2.h5ad') 

In [ ]:
data_DC2

In [ ]:
#plotting 
with plt.rc_context({"figure.figsize": (3, 3)}):
    sc.pl.umap(data_DC2, color=["cDC2_Atlas_Subclusters"], ncols=4, size=8)

In [ ]:
#plotting 
with plt.rc_context({"figure.figsize": (3, 3)}):
    sc.pl.umap(data_DC2, color=["cDC2_A-B"], ncols=4, size=8)

In [ ]:
pd.crosstab(data_DC2.obs["sample_type_major2"], data_DC2.obs["cDC2_Atlas_Subclusters"]).apply(
    lambda r: r / r.sum() * 100, axis=1
).plot.bar(stacked=True, figsize=(4,6))
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)
plt.title("cDC2 Clusters by Sample Type")
plt.tight_layout()  
plt.show()

### Calculate DEGs between cluster 6 and other cDC2B

In [ ]:
# subset to cDC2B
data_DC2B = data_DC2[data_DC2.obs["cDC2_A-B"] == "cDC2B"]

In [ ]:
# check annots
print(data_DC2B.obs['cDC2_Atlas_Subclusters'].value_counts())

In [ ]:
#calculate DEG genes between cluster 6 and all others:
sc.tl.rank_genes_groups(data_DC2B, groupby="cDC2_Atlas_Subclusters", groups=["6"], reference="rest", method="wilcoxon")

In [ ]:
with plt.rc_context({"figure.figsize": (12, 2)}):
    sc.pl.rank_genes_groups_violin(data_DC2B, groups=["6"], n_genes=20, jitter=False, strip=False)

In [ ]:
# save DEGs
DC2_Degs = sc.get.rank_genes_groups_df(data_DC2B, group="6")
DC2_Degs.to_csv("20251016_DC2B_Degs_c6.csv", index=False)

In [ ]:
DC2_Degs.head()

In [ ]:
#plotting 
with plt.rc_context({"figure.figsize": (3, 3)}):
    sc.pl.umap(data_DC2, color=["sample_type_major2"], ncols=4, size=8)

In [ ]:
with plt.rc_context({"figure.figsize": (4, 4)}):
    sc.pl.umap(data_DC2, color="sample_type_major2", groups=["primary tumour"], size=10, title="primary", frameon=False, legend_loc=None)
    sc.pl.umap(data_DC2, color="sample_type_major2", groups=["metastatic tumour"], size=10, title="mets", frameon=False, legend_loc=None)
    sc.pl.umap(data_DC2, color="sample_type_major2", groups=["healthy"], size=10, title="healthy", frameon=False, legend_loc=None)
    sc.pl.umap(data_DC2, color="sample_type_major2", groups=["ascites"], size=10, title="ascites", frameon=False, legend_loc=None)

# Subset out cDC1 from full DC object. 

In [ ]:
# read in data
data = sc.read_h5ad('20250617_AllDC_bbknn.h5ad')

In [ ]:
# read in data
data = sc.read_h5ad('20250617_AllDC_bbknn.h5ad')

In [ ]:
#plotting 
with plt.rc_context({"figure.figsize": (3, 3)}):
    sc.pl.umap(data, color=["NR_annotations_simple"], ncols=4, size=8)

In [ ]:
with plt.rc_context({"figure.figsize": (4, 4)}):
    sc.pl.umap(data, color="sample_type_major2", groups=["primary tumour"], size=10, title="primary", frameon=False, legend_loc=None)
    sc.pl.umap(data, color="sample_type_major2", groups=["metastatic tumour"], size=10, title="mets", frameon=False, legend_loc=None)
    sc.pl.umap(data, color="sample_type_major2", groups=["healthy"], size=10, title="healthy", frameon=False, legend_loc=None)
    sc.pl.umap(data, color="sample_type_major2", groups=["ascites"], size=10, title="ascites", frameon=False, legend_loc=None)

In [ ]:
pd.crosstab(data.obs["sample_type_major2"], data.obs["NR_annotations_simple"]).apply(
    lambda r: r / r.sum() * 100, axis=1
).plot.bar(stacked=True, figsize=(4,6))
plt.legend(bbox_to_anchor=(1, 0.5), loc="center left", frameon=False)
plt.title("DC subsets by Sample Type")
plt.tight_layout()  
plt.show()

### Looks like cDC1 are also distinct in ascites, need to read in pre-hvg object, subset out cDC1, repeat clustering, and then caluclate degs:

In [ ]:
# read in data
data_labels = data
data_clean_prehvg = sc.read_h5ad('NR_cleandata.h5ad')